In [ ]:
import numpy as np

from synth_config_generator import (ConfigGenerator, get_uniform_module_distribution, add_confusion, add_split,
                                    get_variable_module_distribution)


In [ ]:
os_labels = [
    'OperatingSystem.Windows',
    'OperatingSystem.Linux',
    'OperatingSystem.MacOS',
    'OperatingSystem.iOS',
    'OperatingSystem.Android',
]

module_names = [x for x in "abcdefghi"]

In [ ]:
gen = ConfigGenerator(base_dir="config/synthesis/full_synth_base",
                      output_dir="config/synthesis",
                      module_names_=module_names,
                      os_labels_=os_labels)

In [ ]:
for n in [0.6, 0.7, 0.8, 0.9]:
    acc_id = f"{int(n * 100)}"
    gen.generate_config(f"uni_{acc_id}",
                        lambda: get_uniform_module_distribution(os_labels, n))

    for split_cnt in [1, 3]:
        gen.generate_config(f"split_{split_cnt}x3_{acc_id}",
                            lambda: add_split(get_uniform_module_distribution(os_labels, n), split_cnt, 3))

    for module_cnt in range(3, len(module_names) + 1, 2):
        for os_cnt in range(2, len(os_labels) + 1, 1):
            gen.generate_config(f"{os_cnt}c_{module_cnt}am_uni_{acc_id}",
                                lambda: get_uniform_module_distribution(os_labels[:os_cnt], n),
                                active_modules_cnt=module_cnt,
                                overlap_mean=module_cnt - 1,
                                class_cnt=os_cnt)

In [ ]:
for module_cnt in range(3, len(module_names) + 1, 2):
    for os_cnt in range(2, len(os_labels) + 1, 1):
        gen.generate_config(f"{os_cnt}c_{module_cnt}am_variable",
                            lambda: get_variable_module_distribution(os_labels[:os_cnt], 1 / os_cnt, 0.9),
                            active_modules_cnt=module_cnt,
                            overlap_mean=module_cnt - 1,
                            class_cnt=os_cnt)

In [ ]:
n = 0.7
acc_id = f"{int(n * 100)}"
for module_cnt in range(3, len(module_names) + 1, 2):
    for split_cnt in range(1, 4, 2):
        for split_size in range(2, 4):
            gen.generate_config(f"{module_cnt}am_split_{split_cnt}x{split_size}_{acc_id}",
                                lambda: add_split(get_uniform_module_distribution(os_labels, n), split_cnt,
                                                  split_size),
                                active_modules_cnt=module_cnt,
                                overlap_mean=module_cnt - 1)

    for conf_cnt in range(1, 4, 2):
        for conf_size in range(2, 4):
            gen.generate_config(f"{module_cnt}am_confused_{conf_cnt}x{conf_size}_{acc_id}",
                                lambda: add_confusion(get_uniform_module_distribution(os_labels, n), conf_cnt,
                                                      conf_size),
                                active_modules_cnt=module_cnt,
                                overlap_mean=module_cnt - 1)


In [ ]:
for i in range(20):
    module_cnt = np.random.randint(3, 10)
    class_cnt = np.random.randint(2, 6)
    print(module_cnt, class_cnt)
    generators = []
    for j in range(class_cnt):
        dist = np.random.choice(['variable', 'uniform', 'chaos'], p=[0.34, 0.33, 0.33])
        if dist == 'variable':
            generators.append(lambda: get_variable_module_distribution(os_labels[:class_cnt], 1 / class_cnt, 0.9))
            print(dist)
        else:
            acc = np.random.randint(100 / class_cnt, 95) / 100
            generators.append(lambda: get_uniform_module_distribution(os_labels[:class_cnt], acc))
            if dist == 'chaos':
                chaos_size = np.random.randint(2, class_cnt) if class_cnt != 2 else class_cnt
                chaos_count = np.random.randint(1, class_cnt)
                chaos_kind = np.random.choice([add_confusion, add_split])
                generators.append(lambda: chaos_kind(get_uniform_module_distribution(os_labels[:class_cnt], acc), chaos_count, chaos_size))
                print(dist, acc, chaos_kind, chaos_count, chaos_size)
            else:
                print(dist, acc)
    print()

    gen.generate_config(f"random_{i}",
                        generators,
                        active_modules_cnt=module_cnt,
                        overlap_mean=module_cnt - 1,
                        class_cnt=class_cnt)
